In [1]:
# installing modules
!pip install openai python-dotenv langchain-community langchain
# python-dotevn to load .env files
# by default .env file will be loaded

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [16]:
# importing module
from openai import OpenAI
from dotenv import load_dotenv
from langchain.prompts import  PromptTemplate
from langchain_community.chat_models import ChatOpenAI


In [3]:
# loading .env file
load_dotenv() # auto load .env file

True

In [9]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="hello how are you ?"
)

print(response.output_text)

Hi! I’m doing well, thanks for asking. How can I help you today?


In [14]:
# prompt template example
myprompt = "tell me a jobe about {topic}"

mytemplate = PromptTemplate(
    input_variables=["topic"],
    template=myprompt
)

# calling
my_final_prompt = mytemplate.format(topic="donkey")
print(my_final_prompt)

tell me a jobe about donkey


In [15]:
# method 1 using openai module
response1 = client.responses.create(
    model="gpt-5",
    input=my_final_prompt
)

print(response1.output_text)

What do you call a donkey with one leg? A wonky donkey.


In [17]:
# method 2 using langchain_community
llm=ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5
)

/tmp/ipython-input-2965849913.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(


In [19]:
# invoke llm
response2 = llm.invoke(my_final_prompt)
print(response2.content)

Donkey caretaker: A donkey caretaker is responsible for the daily care and well-being of donkeys. This includes feeding, grooming, and exercising the animals, as well as monitoring their health and providing any necessary medical care. Donkey caretakers may also be responsible for cleaning and maintaining the animals' living quarters, such as a barn or pasture. Additionally, they may interact with visitors and educate them about donkeys and their care. Overall, a donkey caretaker plays a crucial role in ensuring the happiness and health of these gentle animals.
